# Artificial Neural Network

### Importing the libraries

In [5]:
 import numpy as np
 import pandas as pd
 import tensorflow as tf

In [6]:
tf.__version__

'2.4.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [10]:
dataset=pd.read_csv("Churn_Modelling.csv")
X=dataset.iloc[:,3:-1].values
y=dataset.iloc[:,-1].values

In [13]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [14]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [15]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:, 2]=le.fit_transform(X[:, 2])

In [16]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X=np.array(ct.fit_transform(X))

In [23]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

### Feature Scaling

In [30]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [31]:
ann=tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [32]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

### Adding the second hidden layer

In [33]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

### Adding the output layer

In [34]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [35]:
ann.compile(optimizer='adam' ,loss ='binary_crossentropy' ,metrics=['accuracy'])

### Training the ANN on the Training set

In [36]:
ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 957us/step - loss: 0.6125 - accuracy: 0.7001
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4958 - accuracy: 0.7901
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4594 - accuracy: 0.7970
Epoch 4/100
250/250 [==============================] - 0s 987us/step - loss: 0.4467 - accuracy: 0.7978
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4469 - accuracy: 0.7982
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4369 - accuracy: 0.7997
Epoch 7/100
250/250 [==============================] - 0s 994us/step - loss: 0.4274 - accuracy: 0.8100
Epoch 8/100
250/250 [==============================] - 0s 955us/step - loss: 0.4285 - accuracy: 0.8196
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4050 - accuracy: 0.8289
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4177 

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [39]:
ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5


array([[False]])

Therefore, our ANN model predicts that this customer stays in the bank!

### Predicting the Test set results

In [40]:
y_pred=ann.predict(X_test)
y_pred=(y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [44]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[1515   80]
 [ 202  203]]
0.859
              precision    recall  f1-score   support

           0       0.88      0.95      0.91      1595
           1       0.72      0.50      0.59       405

    accuracy                           0.86      2000
   macro avg       0.80      0.73      0.75      2000
weighted avg       0.85      0.86      0.85      2000

